<a href="https://colab.research.google.com/github/zenon10/POC-OCR/blob/main/extract_attribute_vRG20230209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.analyticsvidhya.com/blog/2022/06/fine-tune-bert-model-for-named-entity-recognition-in-google-colab/

In [ ]:
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q
!pip install seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikiann", "fr")
label_names = dataset['train'].features['ner_tags'].feature.names;label_names

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset wikiann downloaded and prepared to /root/.cache/huggingface/datasets/wikiann/fr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

https://huggingface.co/datasets/wikiann/viewer/fr/train


In [ ]:
dataset['train'][0]

{'tokens': ['Saxifrage', 'faux', 'Orpin'],
 'ner_tags': [5, 6, 6],
 'langs': ['fr', 'fr', 'fr'],
 'spans': ['LOC: Saxifrage faux Orpin']}

Data Preprocessing
- Bert expects input in input_ids, token_type_ids and attention_mask format
- The label also requires adjustment due to subword tokenization used by BERT

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
'''We can see that len of input_ids is not matching with ner_tags that's why we require to adjust 
  the labels according to the tokenized output.
  We will use the argument truncation=True (to truncate texts that are bigger than the maximum size 
  allowed by the model) as there is a sequence in data which has length>512'''

#Get the values for input_ids, attention_mask, adjusted labels
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True, truncation=True)
  
  total_adjusted_labels = []
  
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    i = -1
    adjusted_label_ids = []
   
    for word_idx in word_ids_list:
      # Special tokens have a word id that is None. We set the label to -100 so they are automatically
      # ignored in the loss function.
      if(word_idx is None):
        adjusted_label_ids.append(-100)
      elif(word_idx!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = word_idx
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])
        
    total_adjusted_labels.append(adjusted_label_ids)
  
  #add adjusted labels to the tokenized samples
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True, remove_columns=['tokens', 'ner_tags', 'langs', 'spans'])

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset

DatasetDict({
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
})

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

Fine tuning

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForTokenClassification, AdamW

In [ ]:
#check if gpu is present
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_names))
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p

    #select predicted index with maximum logit for each token
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

<ipython-input-10-02b457ce3401>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer

batch_size = 16
logging_steps = len(tokenized_dataset['train']) // batch_size
epochs = 2

training_args = TrainingArguments(
    output_dir="mbert-fine-tune-ner_fr",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=True
    )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/gupta99riya/mbert-fine-tune-ner_fr into local empty directory.


In [ ]:
#fine tune using train method
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  Number of trainable parameters = 177268231
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.292900,0.202770,0.878216,0.893823,0.885951,0.941690
2,0.135500,0.188671,0.899955,0.907770,0.903846,0.949130


Saving model checkpoint to mbert-fine-tune-ner_fr/checkpoint-500
Configuration saved in mbert-fine-tune-ner_fr/checkpoint-500/config.json
Model weights saved in mbert-fine-tune-ner_fr/checkpoint-500/pytorch_model.bin
tokenizer config file saved in mbert-fine-tune-ner_fr/checkpoint-500/tokenizer_config.json
Special tokens file saved in mbert-fine-tune-ner_fr/checkpoint-500/special_tokens_map.json
tokenizer config file saved in mbert-fine-tune-ner_fr/tokenizer_config.json
Special tokens file saved in mbert-fine-tune-ner_fr/special_tokens_map.json
Saving model checkpoint to mbert-fine-tune-ner_fr/checkpoint-1000
Configuration saved in mbert-fine-tune-ner_fr/checkpoint-1000/config.json
Model weights saved in mbert-fine-tune-ner_fr/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in mbert-fine-tune-ner_fr/checkpoint-1000/tokenizer_config.json
Special tokens file saved in mbert-fine-tune-ner_fr/checkpoint-1000/special_tokens_map.json
tokenizer config file saved in mbert-fine-tun

TrainOutput(global_step=2500, training_loss=0.21421359252929686, metrics={'train_runtime': 677.7553, 'train_samples_per_second': 59.018, 'train_steps_per_second': 3.689, 'total_flos': 569930319849792.0, 'train_loss': 0.21421359252929686, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 10000
  Batch size = 16


{'eval_loss': 0.18867136538028717,
 'eval_precision': 0.8999553969669938,
 'eval_recall': 0.9077698294866604,
 'eval_f1': 0.9038457231168949,
 'eval_accuracy': 0.9491301830743971,
 'eval_runtime': 23.9935,
 'eval_samples_per_second': 416.78,
 'eval_steps_per_second': 26.049,
 'epoch': 2.0}

To get the precision/recall/f1 computed for each category for test set, we can apply the same function as before on the result of the predict method

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis=2)
# Remove ignored index (special tokens)
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
results = metric.compute(predictions=true_predictions, references=true_labels)
results

***** Running Prediction *****
  Num examples = 10000
  Batch size = 16


{'LOC': {'precision': 0.9038478949751019,
  'recall': 0.9310762917366163,
  'f1': 0.9172600725869436,
  'number': 10722},
 'ORG': {'precision': 0.8404971517348524,
  'recall': 0.847372084928646,
  'f1': 0.8439206170378715,
  'number': 5746},
 'PER': {'precision': 0.9408619062598301,
  'recall': 0.9313404950957497,
  'f1': 0.936076989280964,
  'number': 6423},
 'overall_precision': 0.8981720986376962,
 'overall_recall': 0.9101393560788082,
 'overall_f1': 0.9041161281923319,
 'overall_accuracy': 0.9483410410588821}

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to mbert-fine-tune-ner_fr
Configuration saved in mbert-fine-tune-ner_fr/config.json
Model weights saved in mbert-fine-tune-ner_fr/pytorch_model.bin
tokenizer config file saved in mbert-fine-tune-ner_fr/tokenizer_config.json
Special tokens file saved in mbert-fine-tune-ner_fr/special_tokens_map.json


In [ ]:
# out = tokenizer("Contrat de Vente International prêt à l'emploi")
# out.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("gupta99riya/mbert-fine-tune-ner_fr")
model = AutoModelForTokenClassification.from_pretrained("gupta99riya/mbert-fine-tune-ner_fr")

##### Process text sample 
from transformers import pipeline

nlp = pipeline('ner', model=model, tokenizer=tokenizer)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [3]:
nlp("Hôpitaux Universitaires de Strasbourg")
#predictions = [label_list[i] for i in preds]

[{'entity': 'LABEL_5',
  'score': 0.5735933,
  'index': 1,
  'word': 'H',
  'start': 0,
  'end': 1},
 {'entity': 'LABEL_5',
  'score': 0.63557523,
  'index': 2,
  'word': '##ô',
  'start': 1,
  'end': 2},
 {'entity': 'LABEL_5',
  'score': 0.6546392,
  'index': 3,
  'word': '##pita',
  'start': 2,
  'end': 6},
 {'entity': 'LABEL_5',
  'score': 0.62116474,
  'index': 4,
  'word': '##ux',
  'start': 6,
  'end': 8},
 {'entity': 'LABEL_4',
  'score': 0.7757413,
  'index': 5,
  'word': 'Univ',
  'start': 9,
  'end': 13},
 {'entity': 'LABEL_4',
  'score': 0.7901702,
  'index': 6,
  'word': '##ers',
  'start': 13,
  'end': 16},
 {'entity': 'LABEL_4',
  'score': 0.79054195,
  'index': 7,
  'word': '##itaires',
  'start': 16,
  'end': 23},
 {'entity': 'LABEL_4',
  'score': 0.80061716,
  'index': 8,
  'word': 'de',
  'start': 24,
  'end': 26},
 {'entity': 'LABEL_4',
  'score': 0.8160277,
  'index': 9,
  'word': 'Strasbourg',
  'start': 27,
  'end': 37}]

In [4]:
nlp("11 mois")

[{'entity': 'LABEL_5',
  'score': 0.44426668,
  'index': 1,
  'word': '11',
  'start': 0,
  'end': 2},
 {'entity': 'LABEL_4',
  'score': 0.60893404,
  'index': 2,
  'word': 'mois',
  'start': 3,
  'end': 7}]

In [ ]:
{k:v for k,v in enumerate(['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])}

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC'}